#Deep Learning con Python
`Autor: Erwing FC 
~erwingforerocastro@gmail.com`

###Multi-input model

Un modelo típico de preguntas y respuestas tiene dos entradas: una pregunta en lenguaje natural y un fragmento de texto (como un artículo de noticias) que proporciona información que se utilizará para responder la pregunta.

In [ ]:
from keras.models import Model
from keras import layers
from keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

#el tamaño de la entrada de la red
#el embedding del texto de entrada
#la capa de LSTM del texto de entrada
text_input = Input(shape=(None,), dtype='int32', name='text')
embedded_text = layers.Embedding(64, text_vocabulary_size)(text_input)
encoded_text = layers.LSTM(32)(embedded_text)

#el tamaño de la entrada de la red
#el embedding de la pregunta 
#la capa de LSTM de la pregunta 
question_input = Input(shape=(None,),dtype='int32',name='question')
embedded_question=layers.Embedding(32,question_vocabulary_size)(question_input)
encoded_question=layers.LSTM(16)(embedded_question)

#unimos ambas codificaciones
concatenated=layers.concatenate([encoded_text,encoded_question],axis=-1)
#agregamos la ultima capa softmax de la respuesta
answer=layers.Dense(answer_vocabulary_size,activation='softmax')(concatenated)

#agregamos la(s) entrada(s) y la ultima capa de la red
net=Model([text_input,question_input],answer)
#compilacion
net.compile(optimizer='rmsprop',
            loss='categorical_crossentropy',
            metrics=['acc'])

Alimentando un modelo de entradas multiples

In [ ]:
import numpy as np
num_samples=1000
max_length=100

text=np.random.randint(1,text_vocabulary_size,size=(num_samples,max_length))
question=np.random.randint(1,question_vocabulary_size,size=(num_samples,max_length))
answers=np.random.randint(0,1,size=(num_samples,answer_vocabulary_size)) #one-hot encoding 

#entrenamiento

net.fit([text,question],answers,epochs=10,batch_size=128) #entrenando con una lista de entradas
net.fit({'text': text, 'question': question}, answers,    #entrenando usando un diccionario de entradas
            epochs=10, batch_size=128)

###Multi-output Model

predecir simultáneamente diferentes propiedades de los datos, como una red que toma como entrada una serie de publicaciones en redes sociales de una sola persona anónima y trata de predecir los atributos de esa persona, como la edad, el género , y nivel de ingresos

In [ ]:
from keras import layers
from keras import Input
from keras.models import Model

vocabulary_size = 50000
num_income_groups = 10

#estructura de la red
posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(256, vocabulary_size)(posts_input)
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

#output o ultima capa de la red
age_prediction = layers.Dense(1, name='age')(x) #edad
income_prediction = layers.Dense(num_income_groups,
                                 activation='softmax',
                                 name='income')(x) #nivel de ingresos
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x) #genero

#agregamos las salidas al modelo
net = Model(posts_input,[age_prediction, income_prediction, gender_prediction])

#compilamos
net.compile(optimizer='rmsprop',
            loss=['mse','categorical_crossentropy','binary_crossentropy'],
            loss_weights=[0.25, 1., 10.]) #pesos para equilibrar los tamaños de las perdidas

#equivalente si las salidas de la red contienen la propiedad name
# model.compile(optimizer='rmsprop',
#               loss={'age': 'mse',
#                     'income': 'categorical_crossentropy',
#                     'gender': 'binary_crossentropy'},
#                 loss_weights={'age': 0.25,
#                               'income': 1.,
#                               'gender': 10.})

#entrenamos
model.fit(posts, [age_targets, income_targets, gender_targets],
                  epochs=10, batch_size=64)

#equivalente si las salidas de la red contienen la propiedad name
# model.fit(posts, {'age': age_targets,
#                   'income': income_targets,
#                   'gender': gender_targets},
#           epochs=10, batch_size=64)

###Siames LSTM

Dos entradas una capa

In [ ]:
from keras import layers
from keras import Input
from keras.models import Model
#LSTM
lstm = layers.LSTM(32)
#las entradas y conexiones con LSTM
left_input = Input(shape=(None, 128))
left_output = lstm(left_input)
right_input = Input(shape=(None, 128))
right_output = lstm(right_input)
#la union
merged = layers.concatenate([left_output, right_output], axis=-1)
#la capa de salida
predictions = layers.Dense(1, activation='sigmoid')(merged)
#le damos al modelo las capas de entrada y de salida
model = Model([left_input, right_input], predictions)
#entrenamos (H)
model.fit([left_data, right_data], targets)

###Xception

In [ ]:
from keras import layers
from keras import applications
from keras import Input
#la capa
xception_base = applications.Xception(weights=None,
include_top=False)
#las entradas
left_input = Input(shape=(250, 250, 3))
right_input = Input(shape=(250, 250, 3))
#la conexion con la capa
left_features = xception_base(left_input)
right_input = xception_base(right_input)
#la union
merged_features = layers.concatenate(
[left_features, right_input], axis=-1)